In [1]:
#!pip install -r requirements.txt

In [2]:
#!pip install -U sentence-transformers

In [18]:
#!pip install -U  langchain_huggingface 

In [19]:
import os
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from typing import List
import shutil
from langchain_huggingface import HuggingFaceEmbeddings

ImportError: cannot import name 'ToolOutputMixin' from 'langchain_core.messages.tool' (/home/liubov/python_env/myenv/lib/python3.12/site-packages/langchain_core/messages/tool.py)

In [4]:
model_path = 'sentence-transformers/all-MiniLM-L6-v2'
DATA_PATH = 'data/books'
CHROMA_PATH = "chroma"

In [5]:
embeddings = HuggingFaceEmbeddings(model_name=model_path)

/tmp/ipykernel_43454/4115711469.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name=model_path)


In [6]:
class MyEmbeddings:
    def __init__(self):
        self.model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self.model.encode(t).tolist() for t in texts]

In [7]:
def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.md")
    documents = loader.load()
    return documents

In [8]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [9]:
documents = load_documents()

In [10]:
chunks = split_text(documents)

Split 1 documents into 801 chunks.
So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.
{'source': 'data/books/alice_in_wonderland.md', 'start_index': 1654}


In [12]:
db = Chroma.from_documents(chunks, embedding=embeddings)
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 20},
    persist_directory="chroma" #директория в которой будут размещаться фаилы
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [15]:
db.search(query = 'How did Alice meet the Mad Hatter?',
          search_type = )

TypeError: VectorStore.search() missing 1 required positional argument: 'search_type'